In [2]:
%load_ext autoreload
%autoreload 2
%load_ext Cython

import sys,time,datetime,copy,subprocess,itertools,pickle,warnings,json,gc,numbers

import numpy as np
import scipy as sp

from matplotlib import pyplot as plt
import matplotlib as mpl

In [3]:
import numpy

class pywatch():
    
    def __init__(self):
        self.times = []
        
    def record(self):
        self.times.append(time.time())
        return self
    
    def print(self):
        self.times.append( time.time() )
        T = np.array(self.times)
        dT = np.ediff1d(T)
        dT = dT * 1e+3
        print(dT)

In [4]:
num_seq = 3
itv = [0,1000]
T = [ np.random.exponential(size=100).cumsum() for _ in range(5) ]
T = [ T_i[T_i<itv[1]] for T_i in T ]
mark = [ np.ones(T_i.shape[0],dtype='i8') * i for i,T_i in enumerate(T) ]

T = np.hstack(T)
mark = np.hstack(mark)
index_sort = np.argsort(T)
T = T[index_sort]
mark = mark[index_sort]
Data = {'T':T, 'mark':mark}

In [12]:
num_seq = 3
itv = [0,100000]
alpha = np.ones((num_seq,num_seq)) * 0.1
beta = np.ones((num_seq,num_seq)) * 10.0
mu = np.ones(num_seq)

Data = {'T':[], 'mark':[]}

t = 0
l_sum = mu.sum()
l_trg = np.zeros((num_seq,num_seq))

while 1:
    
    l_sum = l_trg.sum() + mu.sum()
    step = np.random.exponential(1/l_sum)
    t += step
    l_trg *= np.exp(-beta*step)
    l_sum_next = l_trg.sum() + mu.sum()
    
    if t > itv[1]:
        break
    
    if np.random.rand() < l_sum_next/l_sum:       
        l_cum = np.cumsum( l_trg.sum(axis=0) + mu )
        l_cum = l_cum/l_cum[-1]
        index_event = np.searchsorted(l_cum,np.random.rand())
        Data['T'].append(t)
        Data['mark'].append(index_event)
        l_trg[index_event] += alpha[index_event]*beta[index_event]
        
Data['T'] = np.array(Data['T'])
Data['mark'] = np.array(Data['mark'])
Data['num_seq'] = num_seq

print(len(Data['T']))

428294


In [6]:
%%cython

import numpy as np
cimport numpy as np

def LG_SUM_kernel_cython(np.ndarray[np.float64_t,ndim=1] T, np.ndarray[np.int64_t,ndim=1] mark, int num_seq, np.ndarray[np.float64_t,ndim=2] alpha, np.ndarray[np.float64_t,ndim=2] beta):
    
    cdef int m = num_seq
    cdef int n = T.shape[0]
    
    cdef np.ndarray[np.float64_t,ndim=1] l = np.zeros(n,dtype=np.float64)
    cdef np.ndarray[np.float64_t,ndim=2] dl_a = np.zeros((m,n),dtype=np.float64) 
    cdef np.ndarray[np.float64_t,ndim=2] dl_b = np.zeros((m,n),dtype=np.float64)
    
    cdef np.ndarray[np.float64_t,ndim=2] l_mat = np.zeros((m,m),dtype=np.float64)
    cdef np.ndarray[np.float64_t,ndim=2] dl_mat_a = np.zeros((m,m),dtype=np.float64)
    cdef np.ndarray[np.float64_t,ndim=2] dl_mat_b = np.zeros((m,m),dtype=np.float64)
    
    cdef int i,j,k
    cdef int index1,index2
    cdef double step = 1.0
    
    cdef np.ndarray[np.float64_t,ndim=3] r = np.exp( - beta * np.ediff1d(T).reshape(-1,1,1) )
        
    for i in range(1,n):
        index1 = mark[i-1]
        index2 = mark[i]
        step = T[i] - T[i-1]
        
        for j in range(m):
            l_mat[index1,j] += alpha[index1,j]*beta[index1,j]
            dl_mat_a[index1,j] += beta[index1,j]
            dl_mat_b[index1,j] += alpha[index1,j]
            
        for j in range(m):
            for k in range(m):
                l_mat[j,k] *= r[i-1,j,k]
                dl_mat_a[j,k] *= r[i-1,j,k]
                dl_mat_b[j,k] = dl_mat_b[j,k]*r[i-1,j,k] - l_mat[j,k]*step
        
        for j in range(m):
            l[i] += l_mat[j,index2]
            
        for j in range(m):
            dl_a[j,i] = dl_mat_a[j,index2]
            dl_b[j,i] = dl_mat_b[j,index2]

                
    return [l,{'alpha':dl_a,'beta':dl_b}]

In [7]:
from Quasi_Newton import Quasi_Newton

class estimator():
    
    def set_data(self,Data,itv):
        self.Data = Data
        self.itv = itv
        return self
    
    def set_para(self,para):
        self.para = para
        return self
    
    def fit(self,Data,itv):
        
        num_seq = Data['num_seq']
        self.Data = Data
        self.itv = itv
        
        self.stg = {}
        self.stg['para_list'] =      ['mu','alpha','beta']
        self.stg['para_length'] =    {'mu':num_seq, 'alpha':num_seq*num_seq, 'beta':num_seq*num_seq }
        self.stg['para_exp'] =       {'mu':True, 'alpha':True, 'beta':True }
        self.stg['para_ini'] =       {'mu':np.ones(num_seq)*0.2,  'alpha':np.ones(num_seq*num_seq)*0.2,  'beta':np.ones(num_seq*num_seq)*1.0 }
        self.stg['para_step_Q'] =    {'mu':np.ones(num_seq)*0.2,  'alpha':np.ones(num_seq*num_seq)*0.2,  'beta':np.ones(num_seq*num_seq)*0.2 }
        self.stg['para_step_diff'] = {'mu':np.ones(num_seq)*0.01, 'alpha':np.ones(num_seq*num_seq)*0.01, 'beta':np.ones(num_seq*num_seq)*0.01 }

        [para,L,ste,G_norm,i_loop] = Quasi_Newton(self)
        
        return para
    
    def LG_baseline(self):
        
        m = self.Data['num_seq']
        T = self.Data['T']
        mark = self.Data['mark']
        itv = self.itv
        st,en = itv
        n = T.shape[0]
        
        mu = self.para['mu']
        
        l = mu[mark]
        dl_mu = np.ones(n)
        Int = mu.sum()*(en-st)
        dInt_mu = np.ones(m)*(en-st)
        
        return [l,{'mu':dl_mu},Int,{'mu':dInt_mu}]
    
    def LG_SUM_kernel(self):
        
        m = self.Data['num_seq']
        T = self.Data['T']
        mark = self.Data['mark']
        itv = self.itv
        st,en = itv
        n = T.shape[0]
        
        alpha = self.para['alpha'].reshape(m,m)
        beta = self.para['beta'].reshape(m,m)
        
        t1 = time.time()
        l = np.zeros(n)
        dl_a = np.zeros((m,n)) 
        dl_b = np.zeros((m,n))
        
        l_mat    = np.zeros((m,m))
        dl_mat_a = np.zeros((m,m))
        dl_mat_b = np.zeros((m,m))
        
        for i in range(1,n):
            index1 = mark[i-1]
            index2 = mark[i]

            l_mat[index1] += alpha[index1] * beta[index1]
            dl_mat_a[index1] +=  beta[index1]
            dl_mat_b[index1] += alpha[index1]
            l_mat *= np.exp(-beta*(T[i]-T[i-1]))
            dl_mat_a *= np.exp(-beta*(T[i]-T[i-1]))
            dl_mat_b = dl_mat_b * np.exp(-beta*(T[i]-T[i-1])) - l_mat * (T[i]-T[i-1])
            
            l[i] = l_mat[:,index2].sum()
            dl_a[:,i] = dl_mat_a[:,index2]
            dl_b[:,i] = dl_mat_b[:,index2]
            
        return [l,{'alpha':dl_a,'beta':dl_b}]

    def LG_INT_kernel(self):
        
        m = self.Data['num_seq']
        T = self.Data['T']
        mark = self.Data['mark']
        itv = self.itv
        st,en = itv
        n = T.shape[0]
        
        alpha = self.para['alpha'].reshape(m,m)
        beta = self.para['beta'].reshape(m,m)
        
        t_to_en = (en-T).reshape(-1,1)
        alpha_t = alpha[mark]
        beta_t = beta[mark]
        mark_t = np.array([ mark == i for i in range(m) ]).astype('f8')
        
        Int = ( alpha_t * ( 1 - np.exp(-beta_t*t_to_en) ) ).sum()
        dInt_a = mark_t.dot( 1 - np.exp(-beta_t*t_to_en) )
        dInt_b = mark_t.dot( alpha_t * np.exp(-beta_t*t_to_en) * t_to_en)
            
        return [Int,{'alpha':dInt_a, 'beta':dInt_b}]
        
    def LG(self,para,only_L=False):
        T = self.Data['T']
        mark = self.Data['mark']
        m = self.Data['num_seq']
        n = self.Data['T'].shape[0]
        
        self.para = para
        [l_baseline,dl_baseline,Int_baseline,dInt_baseline] = self.LG_baseline()
        [l_kernel,dl_kernel] = LG_SUM_kernel_cython(T,mark,m,para['alpha'].reshape(m,m),para['beta'].reshape(m,m))
        [Int_kernel,dInt_kernel] = self.LG_INT_kernel()
        
        ### 
        l = l_baseline + l_kernel
        Int = Int_baseline + Int_kernel
        
        mark_t = np.array([ mark == i for i in range(m) ]).astype('f8').transpose()
        
        d_log_l_mu = ( dl_baseline['mu']/l ).dot(mark_t)
        d_log_l_a  = ( dl_kernel['alpha']/l).dot(mark_t) 
        d_log_l_b  = ( dl_kernel['beta']/l ).dot(mark_t)

        L = np.log(l).sum() - Int
        G_mu = d_log_l_mu - dInt_baseline['mu']
        G_a = d_log_l_a - dInt_kernel['alpha']
        G_b = d_log_l_b - dInt_kernel['beta']
        
        return [ L, {'mu':G_mu, 'alpha':G_a.flatten(),'beta':G_b.flatten()} ]

In [13]:
model = estimator().set_data(Data,itv)
model.fit(Data,itv)


{'mu': array([0.99860328, 1.00097523, 0.99590508]),
 'alpha': array([0.10172036, 0.10252492, 0.10221623, 0.10219683, 0.10086128,
        0.09654544, 0.09764189, 0.09867669, 0.09939603]),
 'beta': array([ 9.51611302,  9.90680936,  9.67237313,  9.9784264 , 10.30443144,
        10.07061104, 10.13073323, 10.16511292, 10.19399281])}